# Estudo de Caso: Usando Natural Language para Prever Reviews

## Por: Msc. Eng. Lucas Travi

## Curso: Machine Learning para Data Science - UFSM

## DESCRIÇÃO DO PROBLEMA
Através de algoritmos de Machine Learning podemos não só analisar dados numéricos como também palavras e/ou textos. O filtro de spam do Gmail é um ótimo exemplo do uso de machine learning na detecção de spam ou não dependendo de palavras chaves que aparecem no e-mail. 
Neste estudo de caso analisaremos reviews de um restaurante. Primeiro, limpando o texto e transformando-o em uma matriz que possa ser interpretada e usada para criar um modelo Bayesiano (muito utilizado para este tipo de aplicação). Depois buscaremos formas de otimizar o modelo criado através do uso da técnica de Projeção Aleatória e Análise dos Componentes Principais para reduzir a dimensionalidade do dataset.

In [1]:
# Importar numpy, matplotlib e pandas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importar o dataset como um arquivo tsv
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

In [3]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
print('Quantidade de reviews:',dataset.shape[0])
print('Quantidade de colunas do dataset:',dataset.shape[1])

Quantidade de reviews: 1000
Quantidade de colunas do dataset: 2


## LIMPANDO O TEXTO E CRIANDO A BAG OF WORDS

In [5]:
# Limpar o texto
import re
import nltk
nltk.download('stopwords')
# Biblioteca para tirar palavras que não serão úteis
from nltk.corpus import stopwords
# Biblioteca para transformar as palavras na sua forma mais "simples"
from nltk.stem.porter import PorterStemmer
# Lista para receber as strings geradas pelo loop
corpus = []
# Loop através de todos os reviews
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    # Reunir novamente a lista de palavras geradas na forma de uma string separada por espaços
    review = ' '.join(review)
    corpus.append(review)

# Demonstrar a lista "limpa" gerada após o loop
corpus[:5]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Travi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price']

In [6]:
# Criar o Bag of Words e determinar X e y
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [7]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

A matrix X criada é composta por 1500 colunas e 1000 linhas, onde a célula receberá o valor 0 se a palavra não aparece no review e 1 se ela aparecer.

In [8]:
y

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,

A matriz y possui uma coluna e 1000 linhas, onde a célula recebe o valor 0 se o review foi negativo e 1 se o review foi positivo

Agora usaremos estas duas matrizes para treinar e criar um modelo capaz de prever reviews negativos e positivos de clientes

## TREINANDO E CRIANDO O MODELO

In [9]:
# Dividindo o dataset entre treino e teste
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

C:\Users\Travi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# Treinando o modelo com Naive Bayes
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None)

[`Artigo`](https://medium.com/syncedreview/applying-multinomial-naive-bayes-to-nlp-problems-a-practical-explanation-4f5271768ebf) explicando por que o algoritmo Naive Bayes é uma boa escolha para modelos de NLP (Natural Language Processing)

In [11]:
# Realizando as previsões através do modelo criado
y_pred = classifier.predict(X_test)

In [12]:
# Matriz de Confusão
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[55, 42],
       [12, 91]], dtype=int64)

## REDUZINDO A DIMENSIONALIDADE

In [13]:
# Importar projeção aleatória e aplicar à matriz X
from sklearn import random_projection
rp = random_projection.SparseRandomProjection(eps=0.3, random_state=0)
new_X = rp.fit_transform(X)
new_X
# Criar uma variável para garantir que a quantidade de componentes da RP e do PCA sejam os mesmos
n_components = new_X.shape[1]

In [14]:
print('Quantidade de colunas em new_X:',new_X.shape[1])
print('Quanidade de linhas em new_X:',new_X.shape[0])

Quantidade de colunas em new_X: 767
Quanidade de linhas em new_X: 1000


In [15]:
# Dividir as matrizes entre treino e teste novamente
new_X_train, new_X_test, y_train, y_test = train_test_split(new_X, y, test_size = 0.20, random_state = 0)

In [16]:
# Criar o classificador com a nova matriz new_X_train
classifier.fit(new_X_train, y_train)

GaussianNB(priors=None)

In [17]:
# Realizando as previsões através do modelo criado
y_pred = classifier.predict(new_X_test)

In [18]:
# Matriz de Confusão
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[70, 27],
       [38, 65]], dtype=int64)

Através da projeção aleatória foi possível reduzir a quantidade de dimensões na matriz X de 1000 para 767 e mesmo assim obter um modelo que seja capaz de prever pelo menos 3/4 dos reviews corretamente.

In [19]:
# Importar PCA e aplicar à matriz X
from sklearn.decomposition import PCA
pca = PCA(n_components=n_components, random_state=0)
pca.fit(X)
new_X = pca.transform(X)
new_X

array([[-3.82240358e-01,  7.99509249e-01, -8.97481303e-02, ...,
         5.21691954e-03,  3.65084442e-02, -4.42417233e-02],
       [ 2.47014957e-01, -4.82580180e-02,  7.89097208e-01, ...,
         9.25075177e-04, -5.71446866e-02, -7.11529289e-02],
       [-1.11121154e-01, -1.49990524e-01,  3.79757917e-02, ...,
         1.13020219e-02, -3.19553079e-02,  1.93877223e-02],
       ...,
       [-3.96008825e-01, -1.75376890e-02,  1.33459893e-01, ...,
        -2.35902889e-02, -4.43233215e-04,  1.77830153e-02],
       [-2.86666847e-01, -2.75480943e-02,  1.54603331e-02, ...,
        -3.49476676e-03,  2.82202299e-03, -2.48221584e-03],
       [-1.44141816e-01, -1.74256144e-01,  8.69781812e-03, ...,
         8.96065975e-03,  3.05198012e-04,  3.83425301e-03]])

In [20]:
print('Quantidade de colunas em new_X:',new_X.shape[1])
print('Quanidade de linhas em new_X:',new_X.shape[0])

Quantidade de colunas em new_X: 767
Quanidade de linhas em new_X: 1000


In [21]:
# Dividir as matrizes entre treino e teste novamente
new_X_train, new_X_test, y_train, y_test = train_test_split(new_X, y, test_size = 0.20, random_state = 0)

In [22]:
# Criar o classificador com a nova matriz new_X_train
classifier.fit(new_X_train, y_train)

GaussianNB(priors=None)

In [23]:
# Realizando as previsões através do modelo criado
y_pred = classifier.predict(new_X_test)

In [24]:
# Matriz de Confusão
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[47, 50],
       [35, 68]], dtype=int64)

Utilizando PCA foi obtido um índice de acerto de aproximadamente 57.5% na previsão dos reviews, um aproveitamento inferior ao obtido com a projeção aleatória.
Uma das principais motivações de usar redutores de dimensionalidade é fazer os algoritmos trabalharem com mais facilidade pelo fato de terem menos atributos a serem "aprendidos". No caso que vimos neste estudo de caso foi possível utilizar o método de projeção aleatória e conseguir resultados satisfatório, reduzindo em praticamente 1/4 a nossa quantidade de dimensões. Todavia, pelo fato de não haver um ou mais atributos que se sobressaiam sobre os outros, a técnica de análise dos componentes principais (PCA) acabou não alcançando bons resultados.